In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [5]
commitnumber = de5d43e
molecule = {'co2': 0.0004}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band3c
  atmpro = mls
  band = 3c
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.375563e-24
  molecule = co2
  ng_adju = [0, 0, 0]
  ng_refs = [3, 2, 4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 800
  vmin = 720
  w_diffuse = [(1.7, 1.6, 1.8), (1.8, 1.7), (1.5, 1.6, 1.7, 1.8)]
  wgt = [(0.6, 0.4, 0.7), (0.7, 0.4), (0.3, 0.4, 0.85, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.257034                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.103677
0.000750      2.0  0.256101                  2.0  0.106792
0.001052      3.0  0.265118                  3.0  0.115273
0.001476      4.0  0.273713                  4.0  0.124209
0.002070      5.0  0.280407                  5.0  0.132913
0.002904      6.0  0.284825                  6.0  0.141141
0.004074      7.0  0.286051                  7.0  0.148613
0.005714      8.0  0.283297                  8.0  0.154823
0.008015      9.0  0.275501                  9.0  0.159222
0.011243     10.0  0.261527                 10.0  0.160815
0.015771     11.0  0.240159                 11.0  0.158447
0.022122     12.0  0.210101                 12.0  0.150235
0.031031     13.0  0.169796                 13.0  0.133910
0.043528     14.0  0.117367                 14.0  0.106897
0.061057     15.0  0.076957                 15.0  0.088683
0.085645     16.0  0.092601                 16.0  0.117802
0.120136     17.0  0.146954                 17.0  0.180737
0.168516     18.0  0.190622                 18.0  0.225439
0.236378     19.0  0.225765                 19.0  0.241479
0.331549     20.0  0.263634                 20.0  0.244478
0.465100     21.0  0.320290                 21.0  0.273595
0.652400     22.0  0.429392                 22.0  0.383173
0.915100     23.0  0.567534                 23.0  0.540179
1.283650     24.0  0.551381                 24.0  0.544472
1.800600     25.0  0.419375                 25.0  0.426604
2.525700     26.0  0.317295                 26.0  0.326913
3.542800     27.0  0.243206                 27.0  0.257622
4.969550     28.0  0.187570                 28.0  0.206659
6.970850     29.0  0.146930                 29.0  0.167372
9.778100     30.0  0.115574                 30.0  0.129936
13.715850    31.0  0.087443                 31.0  0.093045
19.239350    32.0  0.061759                 32.0  0.058846
26.987250    33.0  0.044489                 33.0  0.036293
37.855300    34.0  0.034571                 34.0  0.026638
53.100050    35.0  0.024529                 35.0  0.021082
73.887500    36.0  0.013785                 36.0  0.014028
97.662500    37.0  0.007309                 37.0  0.008361
121.437500   38.0  0.003916                 38.0  0.004091
145.212500   39.0 -0.002852                 39.0 -0.004033
168.987500   40.0 -0.010835                 40.0 -0.013270
192.762500   41.0 -0.010326                 41.0 -0.013586
216.537500   42.0 -0.000829                 42.0 -0.004712
240.312500   43.0  0.007953                 43.0  0.003934
264.087500   44.0  0.015659                 44.0  0.011527
287.862500   45.0  0.023010                 45.0  0.018527
311.637500   46.0  0.030148                 46.0  0.025049
335.412500   47.0  0.037102                 47.0  0.031277
359.187500   48.0  0.043872                 48.0  0.037395
382.962500   49.0  0.050441                 49.0  0.043494
406.737500   50.0  0.056727                 50.0  0.049552
430.512500   51.0  0.062609                 51.0  0.055529
454.287500   52.0  0.067931                 52.0  0.061271
478.062500   53.0  0.072609                 53.0  0.066707
501.837500   54.0  0.076709                 54.0  0.071795
525.612500   55.0  0.080384                 55.0  0.076613
549.387500   56.0  0.083769                 56.0  0.081369
573.162500   57.0  0.086944                 57.0  0.086034
596.937500   58.0  0.089966                 58.0  0.090591
620.712500   59.0  0.092879                 59.0  0.095011
644.487500   60.0  0.095721                 60.0  0.099280
668.262500   61.0  0.098535                 61.0  0.103409
692.037500   62.0  0.101394                 62.0  0.107467
715.812500   63.0  0.104389                 63.0  0.111514
739.587500   64.0  0.107610                 64.0  0.115688
76

# Fluxes by Layer

CRD                           CLIRAD             \
                         flug       fldg      fnetg       flug       fldg   
pressure     level                                                          
0.000000e+00 1     -25.978378   0.000000 -25.978378        NaN        NaN   
1.000000e-08 1            NaN        NaN        NaN -25.673231   0.000001   
6.244000e-04 2     -25.978393   0.000034 -25.978359 -25.673247   0.000025   
8.759000e-04 3     -25.978399   0.000047 -25.978352 -25.673253   0.000035   
1.228600e-03 4     -25.978407   0.000067 -25.978341 -25.673262   0.000049   
1.723400e-03 5     -25.978419   0.000094 -25.978324 -25.673275   0.000069   
2.417400e-03 6     -25.978434   0.000133 -25.978301 -25.673293   0.000097   
3.390900e-03 7     -25.978456   0.000187 -25.978269 -25.673317   0.000138   
4.756500e-03 8     -25.978486   0.000264 -25.978222 -25.673351   0.000196   
6.672000e-03 9     -25.978528   0.000370 -25.978158 -25.673399   0.000279   
9.358900e-03 10    -25.978586   0.000516 -25.978070 -25.673465   0.000396   
1.312780e-02 11    -25.978668   0.000715 -25.977954 -25.673559   0.000562   
1.841450e-02 12    -25.978784   0.000981 -25.977803 -25.673692   0.000794   
2.583020e-02 13    -25.978950   0.001331 -25.977619 -25.673882   0.001115   
3.623230e-02 14    -25.979192   0.001782 -25.977409 -25.674157   0.001556   
5.082340e-02 15    -25.979558   0.002351 -25.977207 -25.674563   0.002147   
7.129060e-02 16    -25.980117   0.003097 -25.977020 -25.675164   0.002963   
1.000000e-01 17    -25.980928   0.004223 -25.976705 -25.676015   0.004215   
1.402710e-01 18    -25.982023   0.006019 -25.976004 -25.677148   0.006210   
1.967600e-01 19    -25.983457   0.008729 -25.974728 -25.678605   0.009175   
2.759970e-01 20    -25.985265   0.012656 -25.972609 -25.680404   0.013241   
3.871000e-01 21    -25.987398   0.018258 -25.969140 -25.682482   0.018537   
5.431000e-01 22    -25.989565   0.026344 -25.963221 -25.684550   0.025661   
7.617000e-01 23    -25.990487   0.038385 -25.952102 -25.685313   0.036347   
1.068500e+00 24    -25.987178   0.055702 -25.931476 -25.681718   0.052386   
1.498800e+00 25    -25.979129   0.075757 -25.903371 -25.673319   0.071743   
2.102400e+00 26    -25.969067   0.095681 -25.873386 -25.662693   0.091623   
2.949000e+00 27    -25.957985   0.116420 -25.841565 -25.650758   0.112476   
4.136600e+00 28    -25.946178   0.138827 -25.807351 -25.637655   0.135618   
5.802500e+00 29    -25.933953   0.163616 -25.770337 -25.623555   0.162304   
8.139200e+00 30    -25.921390   0.191724 -25.729667 -25.608265   0.193347   
1.141700e+01 31    -25.908536   0.223745 -25.684792 -25.591749   0.227288   
1.601470e+01 32    -25.896476   0.259309 -25.637167 -25.575157   0.261377   
2.246400e+01 33    -25.888178   0.298192 -25.589986 -25.562056   0.293236   
3.151050e+01 34    -25.886033   0.343722 -25.542311 -25.555441   0.325518   
4.420010e+01 35    -25.891260   0.400915 -25.490345 -25.556706   0.366829   
6.200000e+01 36    -25.910227   0.471602 -25.438625 -25.573341   0.427920   
8.577500e+01 37    -25.955719   0.555918 -25.399801 -25.620500   0.514590   
1.095500e+02 38    -26.018759   0.639543 -25.379216 -25.689179   0.606818   
1.333250e+02 39    -26.095774   0.727586 -25.368188 -25.775190   0.704351   
1.571000e+02 40    -26.192262   0.816042 -25.376220 -25.883179   0.800981   
1.808750e+02 41    -26.317686   0.910952 -25.406734 -26.022128   0.902552   
2.046500e+02 42    -26.470034   1.034218 -25.435816 -26.188175   1.030334   
2.284250e+02 43    -26.638130   1.199979 -25.438151 -26.371082   1.199968   
2.522000e+02 44    -26.817737   1.401985 -25.415752 -26.566048   1.406015   
2.759750e+02 45    -27.007294   1.635643 -25.371651 -26.771215   1.643649   
2.997500e+02 46    -27.205508   1.898661 -25.306847 -26.985077   1.909693   
3.235250e+02 47    -27.411148   2.189206 -25.221942 -27.206134   2.201304   
3.473000e+02 48    -27.623012   2.505562 -25.117450 -27.432814   2.516079   
3.710750e+02 49    -27

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')